In [1]:
import pandas as pd
from tqdm.auto import tqdm
from utils import prepare_ligand, prepare_protien, get_docking_score

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some PyTorch models, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'torch'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
df = pd.read_csv('./KasA_smm_train.csv')
smiles = df['smiles']

In [3]:
from rdkit.rdBase import BlockLogs

def dock_ligands(ligand_chunk, protien_pdb_path):
    with BlockLogs():
        protien_mol = prepare_protien(protien_pdb_path)
        ligand_mols = [prepare_ligand(ligand_smiles) for ligand_smiles in ligand_chunk]

        scores = []
        for ligand_mol in ligand_mols:
            try:
                scores.append(get_docking_score(protien_mol, ligand_mol))
            except Exception as e:
                scores.append(None)

        return scores

In [4]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [5]:
from functools import partial
func = partial(dock_ligands, protien_pdb_path='./KasA_3285_no_min.pdb')

from multiprocessing import Pool
pool = Pool(10)

smiles_chunked = [chunk for chunk in chunks(smiles, 2)]
iterable = pool.imap(func, smiles_chunked)
scores = [s for s in tqdm(iterable, total=len(smiles_chunked))]